# 3.13 丢弃法
除了前一节介绍的权重衰减以外，深度学习模型常常使用丢弃法（dropout）[1] 来应对过拟合问题。丢弃法有一些不同的变体。本节中提到的丢弃法特指倒置丢弃法（inverted dropout）


根据丢弃法的定义，我们可以很容易地实现它。下面的dropout函数将以drop_prob的概率丢弃NDArray输入X中的元素。

## 3.13.2. 从零开始实现

In [29]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dropout

这是一个实现神经网络dropout功能的函数，它的主要目的是通过随机关闭一些神经元来防止过拟合。具体工作过程如下：

1. **输入检查**：
   - 函数接收两个参数：神经元的输出值X和需要丢弃的概率drop_prob
   - 首先检查丢弃概率是否在0到1之间，这是一个合理性检查

2. **特殊情况处理**：
   - 计算保留概率keep_prob（等于1减去丢弃概率）
   - 如果保留概率是0（即丢弃概率是1），就直接返回一个全是0的张量
   - 这种情况意味着所有神经元都被关闭

3. **随机选择要保留的神经元**：
   - 生成一个和输入X形状相同的随机数矩阵，每个数都在0到1之间
   - 将这些随机数与保留概率比较，小于保留概率的位置标记为True
   - 这样就创建了一个随机的布尔掩码，决定哪些神经元保留，哪些丢弃

4. **输出结果计算**：
   - 将布尔掩码转换为数值（True变成1，False变成0）
   - 将保留的神经元输出值除以保留概率进行放大
   - 这样做是为了保持整体输出的期望值不变
   - 被丢弃的神经元输出为0，保留的神经元输出会相应放大

这个过程就像是在打牌时随机扣掉一些牌，但为了保持游戏平衡，剩下的牌的分值会相应提高。这种随机丢弃的方式可以防止神经网络过分依赖某些特定的神经元，从而提高模型的泛化能力。

In [30]:
def dropout(X, drop_prob):
    # 确保dropout概率在[0,1]范围内
    assert 0 <= drop_prob <= 1
    
    # 计算保留概率
    keep_prob = 1 - drop_prob
    
    # 如果保留概率为0（即丢弃概率为1）
    # 则返回全0张量，相当于完全丢弃
    if keep_prob == 0:
        return tf.zeros_like(X)
    
    # 生成随机掩码（mask）
    # 1. tf.random.uniform生成[0,1)之间的均匀分布随机数
    # 2. 将随机数与keep_prob比较，生成布尔型掩码
    # 3. 小于keep_prob的位置为True，表示保留
    mask = tf.random.uniform(
        shape=X.shape, 
        minval=0, 
        maxval=1) < keep_prob
    
    # 应用dropout
    # 1. 将布尔型mask转换为float32类型（True->1.0, False->0.0）
    # 2. 将输入X转换为float32类型
    # 3. 对保留的值进行缩放（除以keep_prob），以保持期望值不变
    return tf.cast(mask, dtype=tf.float32) * tf.cast(X, dtype=tf.float32) / keep_prob

In [31]:
X = tf.reshape(tf.range(0, 16), shape=(2, 8))
dropout(X, 0)

<tf.Tensor: shape=(2, 8), dtype=float32, numpy=
array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11., 12., 13., 14., 15.]], dtype=float32)>

In [32]:
dropout(X, 0.5)

<tf.Tensor: shape=(2, 8), dtype=float32, numpy=
array([[ 0.,  2.,  0.,  6.,  0., 10., 12.,  0.],
       [ 0.,  0., 20., 22., 24.,  0., 28., 30.]], dtype=float32)>

In [33]:
dropout(X, 1)

<tf.Tensor: shape=(2, 8), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>

### 3.13.2.1. 定义模型参数¶

定义一个三层神经网络的参数结构：

1. **网络结构定义**：
   - 输入层有784个神经元（处理28×28的图像）
   - 两个隐藏层，每层都有256个神经元
   - 输出层有10个神经元（做10分类任务）

2. **第一层参数**：
   - W1是连接输入层和第一隐藏层的权重矩阵（784×256）
   - W1是连接输入层和第一隐藏层的权重矩阵（784×256）
   - 使用标准差为0.01的正态分布初始化权重
   - b1是第一隐藏层的偏置项，初始化为0

3. **第二层参数**：
   - W2是连接两个隐藏层的权重矩阵（256×256）
   - 使用标准差为0.1的正态分布初始化权重
   - b2是第二隐藏层的偏置项，初始化为0

4. **输出层参数**：
   - W3是连接第二隐藏层和输出层的权重矩阵（256×10）
   - 使用截断正态分布初始化权重，避免过大的权重值
   - b3是输出层的偏置项，初始化为0

5. **参数集合**：
   - 将所有参数放入一个列表中，方便统一管理和更新

这个网络结构是一个典型的多层感知机（MLP），用于图像分类任务。通过不同的初始化方式（正态分布、截断正态分布）和不同的标准差值，可以帮助网络更好地收敛和学习。

In [34]:
# 定义神经网络的基本参数
num_inputs, num_outputs = 784, 10  # 输入层784个神经元(28x28图像),输出层10个神经元(10分类)
num_hiddens1, num_hiddens2 = 256, 256  # 两个隐藏层,每层256个神经元

# 第一层参数初始化
# 权重W1使用标准差0.01的正态分布初始化,shape为(784,256)
W1 = tf.Variable(tf.random.normal(stddev=0.01, shape=(num_inputs, num_hiddens1)))
b1 = tf.Variable(tf.zeros(num_hiddens1))  # 偏置b1初始化为0

# 第二层参数初始化
# 权重W2使用标准差0.1的正态分布初始化,shape为(256,256) 
W2 = tf.Variable(tf.random.normal(stddev=0.1, shape=(num_hiddens1, num_hiddens2)))
b2 = tf.Variable(tf.zeros(num_hiddens2))  # 偏置b2初始化为0

# 输出层参数初始化
# 权重W3使用截断正态分布初始化,shape为(256,10)
W3 = tf.Variable(tf.random.truncated_normal(stddev=0.01, shape=(num_hiddens2, num_outputs)))
b3 = tf.Variable(tf.zeros(num_outputs))  # 偏置b3初始化为0

# 将所有参数放入列表统一管理
params = [W1, b1, W2, b2, W3, b3]

### 3.13.2.2. 定义模型


In [35]:
# 定义两个隐藏层的dropout比例
# 第一层dropout较少(20%)以保留更多底层特征 
# 第二层dropout较多(50%)以防止过拟合
drop_prob1, drop_prob2 = 0.2, 0.5

def net(X, is_training=False):
    # 将输入X重塑为二维张量
    # 第一维(-1)为批量大小（自动计算），第二维为输入特征数(784)
    X = tf.reshape(X, shape=(-1, num_inputs))
    
    # 第一个隐藏层
    # 1. 线性变换：tf.matmul(X, W1) + b1
    # 2. ReLU激活函数
    H1 = tf.nn.relu(tf.matmul(X, W1) + b1)
    if is_training:  # 只在训练模式下应用dropout
        H1 = dropout(H1, drop_prob1)  # 对第一层输出使用20%的dropout
    
    # 第二个隐藏层
    # 1. 线性变换：tf.matmul(H1, W2) + b2
    # 2. ReLU激活函数
    H2 = tf.nn.relu(tf.matmul(H1, W2) + b2)
    if is_training:  # 只在训练模式下应用dropout
        H2 = dropout(H2, drop_prob2)  # 对第二层输出使用50%的dropout
    
    # 输出层
    # 1. 线性变换：tf.matmul(H2, W3) + b3
    # 2. softmax激活函数用于多分类
    return tf.math.softmax(tf.matmul(H2, W3) + b3)

### 3.13.2.3. 训练和测试模型

In [36]:
from tensorflow.keras.datasets import fashion_mnist

batch_size=256
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = tf.cast(x_train, tf.float32) / 255 
x_test = tf.cast(x_test,tf.float32) / 255
train_iter = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
test_iter = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

In [37]:
# 描述,对于tensorflow2中，比较的双方必须类型都是int型，所以要将输出和标签都转为int型
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for _, (X, y) in enumerate(data_iter):
        y = tf.cast(y,dtype=tf.int64)
        acc_sum += np.sum(tf.cast(tf.argmax(net(X), axis=1), dtype=tf.int64) == y)
        n += y.shape[0]
    return acc_sum / n

In [38]:
def train_ch3(net: callable, 
              train_iter: tf.data.Dataset, 
              test_iter: tf.data.Dataset, 
              loss: callable, 
              num_epochs: int, 
              batch_size: int,
              params: list = None, 
              lr: float = None, 
              trainer: keras.optimizers.Optimizer = None):
    """训练模型的函数
    
    Args:
        net: 神经网络模型
        train_iter: 训练数据集迭代器
        test_iter: 测试数据集迭代器  
        loss: 损失函数
        num_epochs: 训练轮数
        batch_size: 批量大小
        params: 模型参数列表,默认为None
        lr: 学习率,默认为None
        trainer: 优化器,默认为None
    """
    global sample_grads
    for epoch in range(num_epochs):  # 训练所需的迭代次数
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0  # 初始化训练损失之和、训练准确率之和、样本数
        for X, y in train_iter:  # 遍历训练数据集
            with tf.GradientTape() as tape:  # 记录梯度
                y_hat = net(X, is_training=True)  # 前向传播,启用dropout
                # 计算损失,将标签转换为one-hot编码
                l = tf.reduce_sum(loss(y_hat, tf.one_hot(y, depth=10, axis=-1, dtype=tf.float32)))
            
            grads = tape.gradient(l, params)  # 计算梯度
            if trainer is None:  # 如果没有定义优化器
                sample_grads = grads  # 保存梯度用于观察
                # 使用梯度下降优化参数
                params[0].assign_sub(grads[0] * lr)
                params[1].assign_sub(grads[1] * lr)
            else:
                trainer.apply_gradients(zip(grads, params))  # 使用优化器更新参数

            y = tf.cast(y, dtype=tf.float32)  # 将标签转换为float32类型
            train_l_sum += l.numpy()  # 累加训练损失
            # 计算并累加训练准确率
            train_acc_sum += tf.reduce_sum(tf.cast(tf.argmax(y_hat, axis=1) == tf.cast(y, dtype=tf.int64), dtype=tf.int64)).numpy()
            n += y.shape[0]  # 累加训练样本数
        test_acc = evaluate_accuracy(test_iter, net)  # 评估测试集准确率
        # 打印每轮训练的信息
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

In [39]:
# 设置训练参数
num_epochs = 5
lr = 0.5 
batch_size = 256

# 定义损失函数
loss = tf.losses.CategoricalCrossentropy()

# 训练模型
train_ch3(net, train_iter, test_iter, loss, 
          num_epochs, batch_size, params, lr)

epoch 1, loss 0.0355, train acc 0.557, test acc 0.630
epoch 2, loss 0.0269, train acc 0.619, test acc 0.636


2024-10-28 09:45:57.403376: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


epoch 3, loss 0.0253, train acc 0.635, test acc 0.656
epoch 4, loss 0.0239, train acc 0.664, test acc 0.688
epoch 5, loss 0.0226, train acc 0.687, test acc 0.720


## 3.13.3 简洁实现
在Tensorflow2.0中，我们只需要在全连接层后添加Dropout层并指定丢弃概率。在训练模型时，Dropout层将以指定的丢弃概率随机丢弃上一层的输出元素；在测试模型时（即model.eval()后），Dropout层并不发挥作用。

In [40]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(256, activation='relu'),
    Dropout(0.2),
    keras.layers.Dense(256, activation='relu'), 
    Dropout(0.5),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    x_train,
    y_train,
    epochs=5,
    batch_size=256,
    validation_data=(x_test, y_test),
    validation_freq=1
)


Epoch 1/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6719 - loss: 0.9287 - val_accuracy: 0.8379 - val_loss: 0.4474
Epoch 2/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8418 - loss: 0.4429 - val_accuracy: 0.8577 - val_loss: 0.3885
Epoch 3/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8595 - loss: 0.3919 - val_accuracy: 0.8668 - val_loss: 0.3721
Epoch 4/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8706 - loss: 0.3573 - val_accuracy: 0.8690 - val_loss: 0.3685
Epoch 5/5
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8747 - loss: 0.3413 - val_accuracy: 0.8685 - val_loss: 0.3684


小结
我们可以通过使用丢弃法应对过拟合。
丢弃法只在训练模型时使用。


